[1] 모듈 로딩 <hr>

In [270]:
import pandas as pd
import numpy as np
import os
import re

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

from gensim.models import Word2Vec

import spacy

[2] 데이터 불러오기 <hr>

In [271]:
PATH = '../data/Dataset/'

with open(PATH + '0/' + os.listdir(PATH + '0')[0]) as f:
    print(f.read())

동남아 담당' 北 최희철 부상 베이징 도착…싱가포르행 주목	최 부상, 행선지·방문 목적 질문에는 '묵묵부답'

(베이징=연합뉴스) 김진방 특파원 = 북한이 북미 정상회담 무산 가능성까지 거론하며 강경한 태도를 보이는 가운데 동남아시아 외교를 담당하는 최희철 북한 외무성 부상이 19일 중국 베이징 서우두(首都) 공항에 모습을 드러냈다.

최 부상은 이날 오전 평양발 고려항공 JS151편을 이용해 베이징 서우두 공항에 도착했다.

최 부상은 최종 목적지를 묻는 취재진의 질문에 아무런 답변을 하지 않고, 북한 대사관 관계자들과 함께 공항을 빠져나갔다.

북미 정상회담을 20여 일 앞둔 상황에서 동남아 외교통인 최 부상이 정상회담 준비 등을 위해 회담 개최 예정지인 싱가포르를 방문할 가능성도 제기되고 있다.

최 부상은 지난 3월에도 아세안(ASEAN·동남아시아국가연합) 의장국이기도 한 싱가포르를 방문해 양국관계와 올해 8월 열리는 아세안지역안보포럼(ARF) 의제 등을 논의한 바 있다.

또 지난해 북핵 문제를 두고 북미 간 긴장관계가 형성됐을 때도 ARF에 참석해 아세안을 상대로 여론전을 펼쳤다. 북한의 초청으로 비자이 쿠마르 싱 인도 외교부 국무장관이 방북했을 때도 최 부상은 싱 국무장관을 직접 영접하고, 한반도 문제를 논의하기도 했다.

베이징 소식통은 "최 부상이 대(對)미 외교담당이 아니기 때문에 싱가포르로 갈 가능성이 큰 것은 아니다"며 "만약 싱가포르에 간다면 정상회담과 관련한 지원 작업 준비 등을 위한 것일 가능성이 크다"고 말했다.


In [272]:
def read_txt(folder_num):
    txt_list = []

    for file in os.listdir(PATH + f'{folder_num}'):
        with open(PATH + f'{folder_num}/{file}') as f:
            # 정규 표현식을 활용한 구두점, 특수 기호 제거 (공백은 토큰화를 위해 남겨둠)
            text = f.read()
            regex_pattern = re.compile('[^ㄱ-ㅎ가-힣 ]+')
            pre_text = re.sub(regex_pattern, ' ', text)
            txt_list.append(pre_text)
    
    return txt_list

In [273]:
data_dict = {}
file_num = 8

for i in range(file_num):
    for sent in read_txt(i):
        data_dict[sent] = i

data_dict

{'동남아 담당    최희철 부상 베이징 도착 싱가포르행 주목 최 부상  행선지 방문 목적 질문에는  묵묵부답 베이징 연합뉴스  김진방 특파원   북한이 북미 정상회담 무산 가능성까지 거론하며 강경한 태도를 보이는 가운데 동남아시아 외교를 담당하는 최희철 북한 외무성 부상이  일 중국 베이징 서우두  공항에 모습을 드러냈다 최 부상은 이날 오전 평양발 고려항공  편을 이용해 베이징 서우두 공항에 도착했다 최 부상은 최종 목적지를 묻는 취재진의 질문에 아무런 답변을 하지 않고  북한 대사관 관계자들과 함께 공항을 빠져나갔다 북미 정상회담을  여 일 앞둔 상황에서 동남아 외교통인 최 부상이 정상회담 준비 등을 위해 회담 개최 예정지인 싱가포르를 방문할 가능성도 제기되고 있다 최 부상은 지난  월에도 아세안 동남아시아국가연합  의장국이기도 한 싱가포르를 방문해 양국관계와 올해  월 열리는 아세안지역안보포럼  의제 등을 논의한 바 있다 또 지난해 북핵 문제를 두고 북미 간 긴장관계가 형성됐을 때도  에 참석해 아세안을 상대로 여론전을 펼쳤다  북한의 초청으로 비자이 쿠마르 싱 인도 외교부 국무장관이 방북했을 때도 최 부상은 싱 국무장관을 직접 영접하고  한반도 문제를 논의하기도 했다 베이징 소식통은  최 부상이 대 미 외교담당이 아니기 때문에 싱가포르로 갈 가능성이 큰 것은 아니다 며  만약 싱가포르에 간다면 정상회담과 관련한 지원 작업 준비 등을 위한 것일 가능성이 크다 고 말했다 ': 0,
 '예결위  추경 막바지 심사 진통 여야 충돌 서울 연합뉴스  김남권 기자   국회 예산결산특별위원회는  일 추가경정예산안의 막바지 심사에 돌입했으나 여야 간 이견에 진통을 겪고 있다   예결위는 이날 오전  시 소소위원회를 열고 전날까지 심사에서 보류된 사업  건의 감액 심사를 했다   여야  개 교섭단체의 예결위 간사들만 참석하는 소소위는 심사한 지 한 시간 만에 여야 간 충돌로 정회했다   자유한국당 예결위 간사인 김도읍 의원은  감액할 사업이 많은데 현재 소소위가 정회된

In [274]:
dataDF = pd.DataFrame(data = data_dict.items(), columns = ['text', 'label'])

dataDF['label'].value_counts()

label
7    200
4    199
6    197
1    196
2    196
0    195
3    194
5    194
Name: count, dtype: int64

In [275]:
dataDF

text  label
0     동남아 담당    최희철 부상 베이징 도착 싱가포르행 주목 최 부상  행선지 방문 ...      0
1     예결위  추경 막바지 심사 진통 여야 충돌 서울 연합뉴스  김남권 기자   국회 예...      0
2     외압 논란 항명 사태 산 넘고 물 건넌 권성동 영장 청구 안미현 검사 외압 폭로 파...      0
3     친문 홍영표  문빠에 찍혔다 특검 합의에 문자폭탄 공격 대표적인 친 문재인계인 홍영...      0
4       연일  비난 정상회담 전 경고성 메시지 발신  맥스선더 태영호 등 불만 표시  ...      0
...                                                 ...    ...
1566  단일팀 추진  대한카누연맹  데상트코리아와  년 후원 협약 스포티비뉴스 조형애 기자...      7
1567  올림픽 성공 뒷이야기  서울대   일 이희범 평창 조직위원장 초청 특강 이인환 기자...      7
1568   일 개막 호치민 쿠션월드컵에  명 참가 확정 선수  명 참가 차 예선에  명 출전...      7
1569  스포츠안전재단  대축전에 안전필요성 알려 스타뉴스 채준 기자 스포츠안전재단이 대축전...      7
1570  조한승  커피 향 그윽한  대 바둑 보이겠다 한겨레 맥심커피배 트로피 시상식 열려 ...      7

[1571 rows x 2 columns]

In [276]:
features = dataDF['text']
target = dataDF['label']

X_train, X_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 1,
                                                    stratify = target )

[3] 데이터 전처리 <hr>

In [277]:
# 불용어 처리 (Spacy)

TOKEN_MODEL = 'ko_core_news_lg'

nlp = spacy.load(TOKEN_MODEL)

# Spacy 내장 stopwords가 많지 않아, txt파일로 추가
nlp.Defaults.stop_words

{'가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '것으로',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그것',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '놓',
 '누가 알겠는가',
 '누구',
 '는',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',


In [278]:
# 깃허브에 공유되어 있는 한국어 불용어 리스트
STOPWORDS_PATH = '../data/stopwords-ko.txt'

with open(STOPWORDS_PATH) as f:
    stopwords = f.read().split('\n')

stopwords

['가',
 '가까스로',
 '가령',
 '각',
 '각각',
 '각자',
 '각종',
 '갖고말하자면',
 '같다',
 '같이',
 '개의치않고',
 '거니와',
 '거바',
 '거의',
 '것',
 '것과 같이',
 '것들',
 '것으로',
 '게다가',
 '게우다',
 '겨우',
 '견지에서',
 '결과에 이르다',
 '결국',
 '결론을 낼 수 있다',
 '겸사겸사',
 '고',
 '고려하면',
 '고로',
 '곧',
 '공동으로',
 '과',
 '과연',
 '관계가 있다',
 '관계없이',
 '관련이 있다',
 '관하여',
 '관한',
 '관해서는',
 '구',
 '구체적으로',
 '구토하다',
 '그',
 '그들',
 '그때',
 '그래',
 '그래도',
 '그래서',
 '그러나',
 '그러니',
 '그러니까',
 '그러면',
 '그러므로',
 '그러한즉',
 '그런 까닭에',
 '그런데',
 '그런즉',
 '그럼',
 '그럼에도 불구하고',
 '그렇게 함으로써',
 '그렇지',
 '그렇지 않다면',
 '그렇지 않으면',
 '그렇지만',
 '그렇지않으면',
 '그리고',
 '그리하여',
 '그만이다',
 '그에 따르는',
 '그위에',
 '그저',
 '그중에서',
 '그치지 않다',
 '근거로',
 '근거하여',
 '기대여',
 '기점으로',
 '기준으로',
 '기타',
 '까닭으로',
 '까악',
 '까지',
 '까지 미치다',
 '까지도',
 '꽈당',
 '끙끙',
 '끼익',
 '나',
 '나머지는',
 '남들',
 '남짓',
 '너',
 '너희',
 '너희들',
 '네',
 '넷',
 '년',
 '논하지 않다',
 '놀라다',
 '누가 알겠는가',
 '누구',
 '는',
 '다른',
 '다른 방면으로',
 '다만',
 '다섯',
 '다소',
 '다수',
 '다시 말하자면',
 '다시말하면',
 '다음',
 '다음에',
 '다음으로',
 '단지',
 '답다',
 '당신',
 '당장',
 '대로 하다',
 '대하면',
 '대하여',
 '대해 말하

In [279]:
type(nlp.Defaults.stop_words)

set

In [280]:
# Spacy의 내장 불용어 리스트가 set이므로 중복을 신경쓰지 않고, 바로 추가
for word in stopwords:
    nlp.Defaults.stop_words.add(word)
    
    # 불용어 리스트에 단어를 넣어도, 속성 변경을 하지 않으면 불용어 처리를 하지 않음
    nlp.vocab[word].is_stop = True

In [281]:
# 커스텀 데이터셋 생성
class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.n_rows = feature.shape[0]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        return self.feature.iloc[index], self.label.iloc[index]

In [282]:
trainDS = TextDataset(X_train, y_train)
testDS = TextDataset(X_test, y_test)

In [283]:
for feature, label in trainDS:
    print(feature, label)
    break

  화성에 헬기 보낸다 라이트형제 같은 실험 로스앤젤레스 연합뉴스  옥철 특파원   미국항공우주국 이 화성의 대기권역 내 낮은 고도에서 날아다니며 지표면을 관측할 수 있는 소형헬기를 보내겠다는 프로젝트를 발표했다 일 현지시간 에 따르면 는  년 이내에 플로리다 케이프 커내버럴에서 화성 헬리콥터를 붉은 행성 화성 을 향해 쏘아 올릴 것이라고 밝혔다 화성 헬기는 기존의 로버 랜더스 걸어 다니는 관측기기 와 함께 임무를 수행하게 될 것으로 보인다 과학자들에게는 헬기가 새로운  화성의 뷰포인트 를 제공하게 될 것으로 는 기대했다 이는 인류가 지구 밖에서 로켓이나 위성이 아닌 비행체를 처음으로 움직이게 하는 시도가 될 것이라고 은 예상했다 부국장 토머스 저버켄은  라이트 형제가  년 전 지구 상에서 지속해서 날 수 있는 능력을 인류에 제공한 것처럼 또 다른 미국의 개척자들이 또 하나의 세계에서 같은 이상을 실현하는 실험이 될 것 이라고 말했다 화성 대기권에서 날게 될 헬기는 실제 헬기보다는 훨씬 작은 모형 크기가 될 수도 있다  기존의 탐사업무를 수행할 로버가 헬기를 싣고 가서 대기권역 안에서 띄우는 방식으로 임무가 전개된다 헬기는 태양광을 받아 리튬이온 배터리를 충전하는 식으로 작동한다 헬기 프로펠러는 분당  천 회 회전해야 할 정도로 많은 분당 회전수 을 필요로 한다  지구 상에서보다는  배 이상 빠른 속도의 회전력이 요구된다고 는 설명했다 화성의 대기 밀도는 지구의   수준에 불과하기 때문에 이런 회전력으로도 헬기를 조금밖에 띄울 수 없을 것이라고 는 관측했다  지구 상공에서  만 피트 높이로 띄우는 양력이 요구된다는 것이다 는 이달 초 캘리포니아 주 반덴버그 공군기지에서 화성의 지각구조와 열 분포 등 내부 연구에 주력할 탐사선  인사이트 를 실은 아틀라스  로켓을 발사하는 데 성공했다  5


In [284]:
# 토큰화 제너레이터 생성 (메모리 터짐 방지)

def generateToken(dataset):
    for text, label in dataset:
        token_list = []
        doc = nlp(text)

        for token in doc:
            # 불용어, 구두점 처리
            if (not token.is_punct) and (not token.is_stop) and (not token.is_space):
                token_list.append(str(token))
        yield token_list

In [285]:
# 제너레이터는 한번 사용하고 난 후 다시 불러줘야 한다.
train_token_generator = generateToken(trainDS)
test_token_generator = generateToken(testDS)

In [286]:
# 한번에 많은 양의 데이터를 출력하면, Visual Studio가 꺼지므로, 조금만 출력하기 위한 함수
def check_data(data):
    i = 0
    for _ in data:
        print(_)
        i += 1
        if i == 10:
            break

In [287]:
check_data(train_token_generator)

['화성에', '헬기', '보낸다', '라이트형제', '같은', '실험', '로스앤젤레스', '연합뉴스', '옥철', '특파원', '미국항공우주국', '화성의', '대기권역', '내', '낮은', '고도에서', '날아다니며', '지표면을', '관측할', '소형헬기를', '보내겠다는', '프로젝트를', '발표했다', '현지시간', '따르면', '이내에', '플로리다', '케이프', '커내버럴에서', '화성', '헬리콥터를', '붉은', '행성', '화성', '향해', '쏘아', '올릴', '것이라고', '밝혔다', '화성', '헬기는', '기존의', '로버', '랜더스', '걸어', '다니는', '관측기기', '임무를', '수행하게', '될', '보인다', '과학자들에게는', '헬기가', '새로운', '화성의', '뷰포인트', '제공하게', '될', '기대했다', '이는', '인류가', '지구', '밖에서', '로켓이나', '위성이', '아닌', '비행체를', '처음으로', '움직이게', '하는', '시도가', '될', '것이라고', '은', '예상했다', '부국장', '토머스', '저버켄은', '라이트', '형제가', '전', '지구', '상에서', '지속해서', '날', '능력을', '인류에', '제공한', '것처럼', '미국의', '개척자들이', '하나의', '세계에서', '같은', '이상을', '실현하는', '실험이', '될', '이라고', '말했다', '화성', '대기권에서', '날게', '될', '헬기는', '실제', '헬기보다는', '작은', '모형', '크기가', '될', '수도', '기존의', '탐사업무를', '수행할', '로버가', '헬기를', '싣고', '가서', '대기권역', '안에서', '띄우는', '방식으로', '임무가', '전개된다', '헬기는', '태양광을', '받아', '리튬이온', '배터리를', '충전하는', '식으로', '작동한다', '헬기', '프로펠러는', '분당', '천', '회', '회전해야', '할', '정도로', '많

In [231]:
# 단어사전 만들기
token_freqs = {}

for token_list in train_token_generator:
    for token in token_list:
        if token not in token_freqs:
            token_freqs[token] = 1
        else:
            token_freqs[token] += 1

In [232]:
sorted_tokens = sorted(token_freqs.items(), key = lambda x: x[1], reverse = True)

In [233]:
check_data(sorted_tokens)

('말했다', 785)
('기자', 784)
('지난', 766)
('밝혔다', 646)
('서울', 623)
('대한', 611)
('했다', 603)
('만', 578)
('전', 550)
('이라고', 546)


In [234]:
PAD_TOKEN, OOV_TOKEN = 'pad', 'oov'

vocab = {PAD_TOKEN : 0, OOV_TOKEN : 1}

In [235]:
for index, token in enumerate(sorted_tokens, 2):
    vocab[token[0]] = index

In [236]:
check_data(vocab)

pad
oov
말했다
기자
지난
밝혔다
서울
대한
했다
만


In [237]:
vocab['미국항공우주국']

12190

In [238]:
# 수치화 함수 생성
def numbering(generator):
    encoded_data = []

    for token_list in generator:
        L = []
        for token in token_list:
            if token in vocab:
                L.append(vocab[token])
            else:
                L.append(vocab[OOV_TOKEN])
        encoded_data.append(L)
    return encoded_data

In [239]:
train_token_generator = generateToken(trainDS)
test_token_generator = generateToken(testDS)

In [240]:
train_ids = numbering(train_token_generator)
test_ids = numbering(test_token_generator)


In [241]:
check_data(train_ids)
print()
check_data(test_ids)

[6312, 24615, 1, 1, 20, 3166, 4328, 47, 7703, 406, 12190, 22015, 1, 128, 423, 77088, 1, 41364, 27756, 1, 1, 19444, 919, 122, 19, 8353, 7826, 1, 1, 3110, 1, 8042, 20470, 3110, 657, 16264, 3938, 256, 5, 3110, 1, 5051, 1, 1, 3287, 4708, 1, 1, 1, 99, 87, 1, 62064, 134, 22015, 1, 20014, 99, 11433, 144, 10068, 1046, 7470, 1, 1, 251, 1, 612, 28128, 27, 17292, 99, 256, 61, 2106, 1, 54376, 1, 38890, 11432, 10, 1046, 24636, 4523, 310, 1992, 1, 5982, 848, 309, 1, 5928, 3282, 20, 3306, 14178, 28263, 99, 11, 2, 3110, 1, 46535, 99, 1, 333, 1, 439, 77430, 6874, 99, 211, 5051, 1, 8955, 1, 30197, 15232, 2400, 1, 2832, 1, 760, 42476, 1, 1, 1, 500, 1, 9063, 38873, 3659, 1, 24615, 1, 4809, 84, 206, 1, 38, 287, 71, 4809, 1, 15779, 17, 1046, 1, 172, 479, 1, 1, 1, 66, 22015, 3787, 1, 8006, 3232, 1, 1, 30197, 1, 1, 485, 256, 61052, 1046, 7537, 9, 24530, 8244, 1, 1, 1, 37, 411, 221, 2425, 1, 18749, 22015, 1, 3350, 1, 943, 2155, 15451, 1, 52066, 6280, 24344, 71522, 1, 1816]
[13835, 946, 488, 69, 310, 1, 1, 2276

In [242]:
# 패딩을 위한 최대 길이 측정
train_data_length = [len(data) for data in train_ids]
test_data_length = [len(data) for data in test_ids]

if max(train_data_length) >= max(test_data_length):
    MAX_LENGTH = max(train_data_length)

else:
    MAX_LENGTH = max(test_data_length)

MAX_LENGTH = 80


In [243]:
# 패딩 함수 생성
def padding(data_ids):
    for index, data in enumerate(data_ids):
        current_length = len(data)
        if current_length < MAX_LENGTH:
            data_ids[index] = data + ([vocab[PAD_TOKEN]] * (MAX_LENGTH - current_length))
        else:
            data_ids[index] = data[(current_length - MAX_LENGTH) : ]

In [244]:
padding(train_ids)
padding(test_ids)

for data in train_ids[:5]:
    print(len(data), data)

print()

# train 데이터셋으로 만든 단어 사전이라 test 데이터셋에는 oov가 조금 보인다.
for data in test_ids[:5]:
    print(len(data), data)

50 [17, 1046, 1, 172, 479, 1, 1, 1, 66, 22015, 3787, 1, 8006, 3232, 1, 1, 30197, 1, 1, 485, 256, 61052, 1046, 7537, 9, 24530, 8244, 1, 1, 1, 37, 411, 221, 2425, 1, 18749, 22015, 1, 3350, 1, 943, 2155, 15451, 1, 52066, 6280, 24344, 71522, 1, 1816]
50 [21863, 3537, 1988, 673, 7386, 44, 333, 5812, 241, 976, 420, 1617, 2, 1741, 32, 4737, 191, 197, 13891, 7, 5173, 24665, 1, 1, 536, 1, 1, 5327, 874, 8993, 316, 6328, 6125, 83, 16193, 29, 1, 909, 26099, 363, 1219, 252, 1, 14375, 536, 8395, 8993, 31011, 101, 50]
50 [1, 1, 51325, 11180, 5153, 28846, 1, 770, 1979, 1, 436, 85, 411, 221, 1, 1, 3068, 1270, 1, 14634, 20854, 548, 19256, 1, 256, 7339, 144, 408, 9691, 3068, 4704, 44343, 17660, 23993, 913, 1, 1, 149, 1717, 1, 54364, 9858, 40104, 13959, 531, 9691, 7047, 1, 1977, 574]
50 [1, 26158, 1470, 83, 45059, 1, 10481, 285, 209, 5465, 1, 1, 29511, 90, 14113, 7163, 55640, 1, 1, 2043, 7818, 5975, 2861, 1, 174, 107, 459, 1, 1, 1, 1, 907, 1, 1, 1, 53449, 23229, 90, 74922, 55, 7, 2826, 341, 485, 504, 111,

In [245]:
# 커스텀 데이터셋 생성
class TextDataset2(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label = label
        self.n_rows = feature.shape[0]

    def __len__(self):
        return self.n_rows
    
    def __getitem__(self, index):
        return torch.LongTensor(self.feature.iloc[index]), torch.LongTensor(self.label.iloc[index])

In [246]:
# 수치화를 시긴 데이터로 데이터셋 생성
train_df = pd.DataFrame(train_ids)
y_train = pd.DataFrame(y_train)
test_df = pd.DataFrame(test_ids)
y_test = pd.DataFrame(y_test)

train_dataset = TextDataset2(train_df, y_train)
test_dataset = TextDataset2(test_df, y_test)

In [247]:
test_df, y_test

(       0      1      2      3      4      5      6      7      8      9   ...  \
 0    2638     79     25    749     79  20600      1      1    842      1  ...   
 1       1   7636  32890   1356      1  29511   5371      1     71     60  ...   
 2    2459    897    123  23685   9381  11999  12704   7056    184   7568  ...   
 3    4199  60436    891  34974      1   8409      1  22121  52868    445  ...   
 4       1      1  41826   2101  35233  37734    343      1   3216  16472  ...   
 ..    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...  ...   
 310     1    500  10103   1344   2158      1   9223  37068      1  25146  ...   
 311    40     25      1      1   1150   2050      1   1087      5    284  ...   
 312   291   1330  29197  18902  48155    497  48104  48156  12368  24513  ...   
 313     1  12654   7267  80429      1     10      1  43160    392   4561  ...   
 314     1    195    704   2226   1959   1805  13801      1     22   2086  ...   
 
         40   

In [248]:
for text, label in test_dataset:
    print(label)
    break

tensor([7])


In [249]:
# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size = 16, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 16, shuffle = False)

[4] 모델 <hr>

In [250]:
train_token_generator = generateToken(trainDS)
test_token_generator = generateToken(testDS)

In [251]:
# word2vec에 넣으려면 여러번의 에포크를 거쳐야 하기 때문에 제너레이터 대신 리스트로 바꿔준다.
train_token_list = []

for token in train_token_generator:
    train_token_list.append(token)

train_token_list[:5]

[['화성에',
  '헬기',
  '보낸다',
  '라이트형제',
  '같은',
  '실험',
  '로스앤젤레스',
  '연합뉴스',
  '옥철',
  '특파원',
  '미국항공우주국',
  '화성의',
  '대기권역',
  '내',
  '낮은',
  '고도에서',
  '날아다니며',
  '지표면을',
  '관측할',
  '소형헬기를',
  '보내겠다는',
  '프로젝트를',
  '발표했다',
  '현지시간',
  '따르면',
  '이내에',
  '플로리다',
  '케이프',
  '커내버럴에서',
  '화성',
  '헬리콥터를',
  '붉은',
  '행성',
  '화성',
  '향해',
  '쏘아',
  '올릴',
  '것이라고',
  '밝혔다',
  '화성',
  '헬기는',
  '기존의',
  '로버',
  '랜더스',
  '걸어',
  '다니는',
  '관측기기',
  '임무를',
  '수행하게',
  '될',
  '보인다',
  '과학자들에게는',
  '헬기가',
  '새로운',
  '화성의',
  '뷰포인트',
  '제공하게',
  '될',
  '기대했다',
  '이는',
  '인류가',
  '지구',
  '밖에서',
  '로켓이나',
  '위성이',
  '아닌',
  '비행체를',
  '처음으로',
  '움직이게',
  '하는',
  '시도가',
  '될',
  '것이라고',
  '은',
  '예상했다',
  '부국장',
  '토머스',
  '저버켄은',
  '라이트',
  '형제가',
  '전',
  '지구',
  '상에서',
  '지속해서',
  '날',
  '능력을',
  '인류에',
  '제공한',
  '것처럼',
  '미국의',
  '개척자들이',
  '하나의',
  '세계에서',
  '같은',
  '이상을',
  '실현하는',
  '실험이',
  '될',
  '이라고',
  '말했다',
  '화성',
  '대기권에서',
  '날게',
  '될',
  '헬기는',
  '실제',
  '헬기보다는',
  '작은',
  '모형',
  '크기가',

In [252]:
type(train_token_list[0][10])

str

In [253]:
## Word2Vec 모델 생성 및 저장
MODEL_PATH = '../models/'

word2vec = Word2Vec(
    sentences = train_token_list,
    vector_size = 128,
    window = 5,
    min_count = 1,
    sg = 0,
    epochs = 30,
    max_final_vocab = None
)

word2vec.save(MODEL_PATH + 'word2vec.model')

In [254]:
## 생성된 모델 확인
word2vec = Word2Vec.load(MODEL_PATH + 'word2vec.model')

word = '연기'

print(word2vec.wv[word])
print(word2vec.wv.most_similar(word, topn = 5))
print(word2vec.wv.similarity(w1 = word, w2 = '최근'))

[ 0.26443797 -1.3041365   0.38027045  0.7050412   0.6553213  -0.75143504
  0.65654415 -0.27132487 -0.38343892  0.6370621   1.2585199   0.19371116
  0.20239122 -0.21698575  0.19947799  0.49884894 -0.17433588  0.76832527
  0.1634839  -0.16785422  0.28816423  0.06218372 -0.34462795 -0.820281
 -1.1620951   0.04186847 -0.3498842   0.5058852  -0.29205066  0.05745717
 -0.56051856  0.28869903  0.12477758  1.0365256  -0.7451087   0.5363046
  0.35930428 -0.9721925   0.36603883 -0.5381083   0.4372222   0.6472748
  0.15791261 -0.82873964  1.0698997  -0.4135179   0.01311741  0.06879352
 -0.21318142  0.19167557  0.6685357  -1.0282592  -0.0774036   0.35533562
 -0.69516176 -0.50685316  0.54334104 -0.6196777  -0.19320877  0.57730556
  0.37073228  0.5134846  -0.21608546 -0.18806249 -0.42002243 -1.3363512
 -0.17202026  0.04282923 -0.06990165 -0.4579823  -0.38710773  0.6672407
 -1.0342107  -0.5564921  -0.04929825 -0.8643361   0.55119294  0.28687343
 -0.3611516   0.8126741  -0.48779228 -0.91091883  0.81474

In [255]:
# word2vec의 vector_size와 embedding_dim의 크기가 같아야 되는거 같다.

n_vocab = len(vocab)
hidden_dim = 64
embedding_dim = 128
n_layers = 2

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

In [256]:
check_data(vocab.items())

('pad', 0)
('oov', 1)
('말했다', 2)
('기자', 3)
('지난', 4)
('밝혔다', 5)
('서울', 6)
('대한', 7)
('했다', 8)
('만', 9)


In [257]:
check_data(word2vec.wv.key_to_index)

말했다
기자
지난
밝혔다
서울
대한
했다
만
전
이라고


In [258]:
# 임베딩 - word2vec의 max_final_vocab을 제한해 버리면,
# sentences 파라미터에 들어가는 토큰 수가 클 수록, 임베딩이 되지 않는 경우가 생김
# 이번엔 제한 없이 했지만, 단어 사전 수를 제한해서 빈도 수가 적은 애들은 
# 메모리를 생각해서 전부 oov로 전환 시키는 것이 좋을 듯 하다.

init_embeddings = np.zeros((n_vocab, embedding_dim))

for token, index in vocab.items():
    if token not in [PAD_TOKEN, OOV_TOKEN]:
        init_embeddings[index] = word2vec.wv[token]

embedding_layer = nn.Embedding.from_pretrained(
    torch.tensor(init_embeddings, dtype = torch.float32)
)

In [259]:
check_data(init_embeddings)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0.]
[-1.55094898 -0.87952375  2.77873063  3.18257856 -4.58267975 -2.46718431
  0.53077638  3.32931352 -1.22878683 -2.31863117  2.44623232  0.29160661
 -0.63932163  1.80040264 -1.50116181 -0.59534758  2.09213662  0.26875702

In [265]:
# 문장 분류 모델 

class SentenceClassifier(nn.Module):
    def __init__(self,
                 n_vocab,
                 hidden_dim,
                 embedding_dim,
                 n_layers,
                 dropout = 0.5,
                 # 양방향은 빈칸에 들어갈 단어를 고르는 등, 앞 문장과 뒷 문장이 둘 다 중요할 때 사용
                 bidirectional = True,
                 model_type = 'lstm',
                 pretrained_embedding = None
                 ):
        super().__init__()

        if pretrained_embedding is not None:
            self.embedding = nn.Embedding.from_pretrained(
                torch.tensor(pretrained_embedding, dtype = torch.float32)
            )
        else:
            self.embedding = nn.Embedding(
                num_embeddings = n_vocab,
                embedding_dim = embedding_dim,
                padding_idx = 0
        )
        
        if model_type == 'rnn':
            self.model = nn.RNN(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                bidirectional = bidirectional,
                dropout = dropout,
                batch_first = True
            )
        elif model_type == 'lstm':
            self.model = nn.LSTM(
                input_size = embedding_dim,
                hidden_size = hidden_dim,
                num_layers = n_layers,
                bidirectional = bidirectional,
                dropout = dropout,
                batch_first = True
            )
        
        if bidirectional:
            self.classifier = nn.Sequential(
                nn.Linear(hidden_dim * 2, 100),
                nn.ReLU(),
                nn.Linear(100, 50),
                nn.Dropout(dropout)
                nn.ReLU(),
                nn.Linear(50, 8)
            )
        
        else:
            self.classifier = nn.Sequential(
                nn.Linear(hidden_dim, 100),
                nn.ReLU(),
                nn.Linear(100, 50),
                nn.Dropout(dropout),
                nn.ReLU(),
                nn.Linear(50, 8)
            )
        self.dropout = nn.Dropout(dropout)

    def forward(self, inputs):
        embeddings = self.embedding(inputs)
        output, _ = self.model(embeddings)
        last_output = output[:, -1, :]
        last_output = self.dropout(last_output)
        logits = self.classifier(last_output)
        return logits

SyntaxError: invalid syntax (3097719415.py, line 53)

In [266]:
def train(model, datasets, criterion, optimizer, device, interval):
    model.train()
    losses = list()

    for step, (input_ids, labels) in enumerate(datasets):
        input_ids = input_ids.to(device)
        labels = labels.to(device).squeeze(1)

        logits = model(input_ids)
        loss = criterion(logits, labels)
        losses.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % interval == 0:
            print(F"Train Loss {step} : {np.mean(losses)}")

def test(model, datasets, criterion, device):
    model.eval()
    losses = list()
    corrects = list()

    with torch.no_grad():
        for step, (input_ids, labels) in enumerate(datasets):
            input_ids = input_ids.to(device)
            labels = labels.to(device).squeeze(1)

            logits = model(input_ids)
            loss = criterion(logits, labels)
            losses.append(loss.item())
            yhat = torch.argmax(logits, dim = 1)
            corrects.extend(
                torch.eq(yhat, labels).cpu().tolist()
            )

    print(f"Val Loss : {np.mean(losses)}, Val Accuracy : {np.mean(corrects)}")


In [267]:
# for step, (input_ids, labels) in enumerate(train_loader):
#     print(step, torch.argmax(classifier(input_ids), dim = 1), labels.squeeze(1))


In [268]:
check_data(test_loader)

[tensor([[ 2638,    79,    25,   749,    79, 20600,     1,     1,   842,     1,
            79,    25,    30,    84,    41,   520,    84,    41,  5263,  1521,
            84,    41,  2905,    79,  6546,  7896,     1,     1,    22,    77,
           175,   263,     1,     1,    79,    25,   749,    79,     1,     1,
           832,  5759,    79, 31834,   221, 28207,     9,     1,   964,  8258],
        [    1,  7636, 32890,  1356,     1, 29511,  5371,     1,    71,    60,
             1,    17,  1499, 61001,  2011,   687,     1,    29, 63577,   184,
            37,  1530,   453, 48041,  1899,  4321, 52967,  1237,   687,     1,
          4594,   510,  7665,     1,    99,  2058,    23,     2,   284, 14950,
          9432,   397,     1,  4585, 12332,    28,     1,   199,    23,     2],
        [ 2459,   897,   123, 23685,  9381, 11999, 12704,  7056,   184,  7568,
           318,   203,  1251,   170,  2089,     1,   152,   741,  3259,    36,
         13233,  4345,   191, 18876,     1,    98

In [269]:
classifier = SentenceClassifier(
    n_vocab = n_vocab, hidden_dim = hidden_dim, embedding_dim = embedding_dim,
    n_layers = n_layers, pretrained_embedding = init_embeddings
).to(DEVICE)

criterion = nn.CrossEntropyLoss().to(DEVICE)
optimizer = optim.Adam(classifier.parameters(), lr = 0.001)

EPOCH = 100
interval = 20
 
for epoch in range(EPOCH):
    train(classifier, train_loader, criterion, optimizer, DEVICE, interval)
    test(classifier, test_loader, criterion, DEVICE)
    print()

Train Loss 0 : 2.14020037651062
Train Loss 20 : 2.086204210917155
Train Loss 40 : 2.081265257626045
Train Loss 60 : 2.0778258511277494
Val Loss : 1.9623029232025146, Val Accuracy : 0.23492063492063492

Train Loss 0 : 1.96434485912323
Train Loss 20 : 2.0112118550709317
Train Loss 40 : 1.9448198399892667
Train Loss 60 : 1.8928201882565607
Val Loss : 1.9499213457107545, Val Accuracy : 0.28253968253968254

Train Loss 0 : 2.18914794921875
Train Loss 20 : 1.7783480031149728
Train Loss 40 : 1.7570083228553213
Train Loss 60 : 1.7682772894374659
Val Loss : 1.8903540551662446, Val Accuracy : 0.2984126984126984

Train Loss 0 : 1.6127687692642212
Train Loss 20 : 1.7356466679345994
Train Loss 40 : 1.7331295275106662
Train Loss 60 : 1.7143526976225807
Val Loss : 1.6756514191627503, Val Accuracy : 0.3873015873015873

Train Loss 0 : 1.9540956020355225
Train Loss 20 : 1.585786325590951
Train Loss 40 : 1.5721311104006883
Train Loss 60 : 1.5716583386796419
Val Loss : 1.6445275664329528, Val Accuracy : 0.

정확도가 안오르노